# 13.영화 추천 엔진 만들기

In [8]:
# MovieLens data 가져오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("source/u.data", sep="\t", header=None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))

for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=42)
ratings_train.shape, ratings_test.shape

((631, 1682), (312, 1682))

### 사용자 간 유사도 행렬 생성

In [9]:
# 코사인 유사도
from sklearn.metrics.pairwise import cosine_distances

cosine_distances(ratings_train)

array([[0.        , 0.63524236, 0.55753769, ..., 0.97989359, 0.66892071,
        0.74361482],
       [0.63524236, 0.        , 0.57364745, ..., 0.93305581, 0.72660686,
        0.77662732],
       [0.55753769, 0.57364745, 0.        , ..., 0.93324244, 0.74575627,
        0.77679874],
       ...,
       [0.97989359, 0.93305581, 0.93324244, ..., 0.        , 0.95146572,
        0.94857492],
       [0.66892071, 0.72660686, 0.74575627, ..., 0.95146572, 0.        ,
        0.8801978 ],
       [0.74361482, 0.77662732, 0.77679874, ..., 0.94857492, 0.8801978 ,
        0.        ]])

- 코사인 유사도: 내적 공간의 두 벡터의 사이각에 대한 코사인을 측정해 유사도를 나타내는 척도

### 코사인 유사도

- 내적 공간의 두 벡터의 사이각에 대한 코사인을 측정해 유사도를 나타내는 척도

- cos(0) == 1, cos(θ) < 1

In [11]:
distances = 1 - cosine_distances(ratings_train)
distances

array([[1.        , 0.36475764, 0.44246231, ..., 0.02010641, 0.33107929,
        0.25638518],
       [0.36475764, 1.        , 0.42635255, ..., 0.06694419, 0.27339314,
        0.22337268],
       [0.44246231, 0.42635255, 1.        , ..., 0.06675756, 0.25424373,
        0.22320126],
       ...,
       [0.02010641, 0.06694419, 0.06675756, ..., 1.        , 0.04853428,
        0.05142508],
       [0.33107929, 0.27339314, 0.25424373, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.25638518, 0.22337268, 0.22320126, ..., 0.05142508, 0.1198022 ,
        1.        ]])

- 사용자 간의 유사도를 나타냄

- 자기 자신은 1임

- 정방 행렬

### 평가 예측 및 모델의 성능 측정

In [13]:
user_pred = distances.dot(ratings_train) / np.array([np.abs(distances).sum(axis=1)]).T

- np.dot()
    - 행렬 곱
    - 행의 크기가 같아야 연산이 가능함

- np.abs()
    - 음수 제거

- .T
    - 전치 행렬
    - 크기를 맞춰주기 위해서



In [19]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()

    return mean_squared_error(pred, actual)

- nonzero()
    - 0이 아닌 값

- flatten()
    - 1차원으로 변경

In [16]:
# 훈련 data RMSE
np.sqrt(get_mse(user_pred, ratings_train))

2.8075245308903365

In [17]:
# 테스트 data RMSE
np.sqrt(get_mse(user_pred, ratings_test))

2.9870546415652575

단점

- 이 방법은 모든 사용자와 비교함

- 비슷한 사용자를 찾는거에 시간이 많이 걸린다.

### 가장 비슷한 n명을 찾는 비지도 방식의 이웃 검색

In [20]:
from sklearn.neighbors import NearestNeighbors

k = 5
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")

In [21]:
neigh.fit(ratings_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [22]:
top_k_distances, top_k_users = neigh.kneighbors(ratings_train, return_distance=True)

In [25]:
top_k_distances.shape, top_k_users.shape

((631, 5), (631, 5))

In [26]:
top_k_users

array([[  0, 589, 155,  33, 364],
       [  1, 483, 339, 172, 188],
       [  2, 382, 560, 350, 155],
       ...,
       [628, 258, 242, 229, 494],
       [629, 378, 155, 589, 591],
       [630, 495, 201, 417, 603]], dtype=int64)

In [27]:
top_k_distances

array([[0.        , 0.38230161, 0.39990633, 0.40834169, 0.4100445 ],
       [0.        , 0.4625691 , 0.50677921, 0.50811827, 0.50882566],
       [0.        , 0.46538829, 0.48267976, 0.49176259, 0.49265099],
       ...,
       [0.        , 0.5764934 , 0.59340849, 0.64699606, 0.66472075],
       [0.        , 0.60496802, 0.6115226 , 0.62054374, 0.6229481 ],
       [0.        , 0.56320216, 0.60221688, 0.60314589, 0.6400121 ]])

### 선택된 n명의 사용자들의 평가 가중치 합을 사용한 예측 및 모델의 성능 측정

In [30]:
user_pred_k = np.zeros(ratings_train.shape)

In [31]:
for i in range(ratings_train.shape[0]):
    user_pred_k[i, :] = top_k_distances[i].T.dot(ratings_train[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T

In [33]:
user_pred_k.shape

(631, 1682)

In [34]:
# 예측값
user_pred_k

array([[4.25618269, 2.49082621, 0.71654943, ..., 0.        , 0.        ,
        0.        ],
       [3.74418756, 0.        , 2.48873124, ..., 0.        , 0.        ,
        0.        ],
       [3.22293592, 2.98635211, 2.47648118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.07143091, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.73945823, 2.48622549, 1.76969702, ..., 0.        , 0.        ,
        0.        ],
       [1.95357502, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [35]:
# 훈련 RMSE
np.sqrt(get_mse(user_pred_k, ratings_train))

2.0922014531938316

In [36]:
# 테스트 RMSE
np.sqrt(get_mse(user_pred_k, ratings_test))

3.054698791142718

<br/>

### Reference

- https://www.youtube.com/watch?v=Lc5mfCF0mCU